In [ ]:
#Desenvolvido por Ronald Albert, 118021192
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Pesquisa Nacional por Amostra de Domícilios - 2015
A Pesquisa Nacional por Amostra de Domicílios - PNAD investiga anualmente, de forma permanente, características gerais da população, de educação, trabalho, rendimento e habitação e outras, com periodicidade variável, de acordo com as necessidades de informação para o país, como as características sobre migração, fecundidade, nupcialidade, saúde, segurança alimentar, entre outros temas. O levantamento dessas estatísticas constitui, ao longo dos 49 anos de realização da pesquisa, um importante instrumento para formulação, validação e avaliação de políticas orientadas para o desenvolvimento socioeconômico e a melhoria das condições de vida no Brasil.

---

Para trabalhar com a Árvore de Decisão, os dados numéricos que se caracterizam como contínuos serão discretizados, e dados que já são categorizados são substituidos exatamente pela categoria que eles representam, por exemplo, no dataset original a coluna 'Sexo', pode assumir os valores 1 e 0, sendo 1 -> Masculino e 0 -> Feminino, dessa forma esses valores serão substituídos, pelas strings que, de fato, definem tal categoria.

---

Os grupos de renda serão categorizados pelos grupos de classes sociais do IBGE:<br>
E: renda menor do que 1 salario mínimo <br>
D: renda entre 1 salario mínimo e 3 salários mínimos <br>
C: renda entre 3 salários mínimos e 5 salários mínimos <br>
B: renda entre 5 salários mínimos e 15 salários mínimos <br>
A: renda maior do que 15 salários mínimos <br>

In [ ]:
df = pd.read_csv('../input/testes/dados.csv')

def returnGender(g):
    if(g == 1):
        return "Masculino"
    else:
        return "Feminino"
    
def returnUF(u):
    return {
        11: "Rondônia",
        12: "Acre",
        13: "Amazonas",
        14: "Roraima",
        15: "Pará",
        16: "Amapá",
        17: "Tocantis",
        21: "Maranhão",
        22: "Piauí",
        23: "Ceará",
        24: "Rio Grande do Norte",
        25: "Paraíba",
        26: "Pernambuco",
        27: "Alagoas",
        28: "Sergipe",
        29: "Bahia",
        31: "Minas Gerais",
        32: "Espírito Santo",
        33: "Rio de Janeiro",
        35: "São Paulo",
        41: "Paraná",
        42: "Santa Catarina",
        43: "Rio Grande do Sul",
        50: "Mato Grosso do Sul",
        51: "Mato Grosso",
        52: "Goias",
        53: "Distrito Federal"
    }[u]

def returnEthnicGroup(e):
    return {
        0: "Indígena",
        2: "Branco",
        4: "Negro",
        6: "Asiático",
        8: "Pardo",
        9: "Sem Declaração"
    }[e]

def returnGruposDeIdade(i):
    if(i <= 18):
        return "Menor de 18 anos"
    elif(i <= 25):
        return "Entre 18 e 25 anos"
    elif(i <= 40):
        return "Entre 25 e 40 anos"
    elif (i <= 60):
        return "Entre 40 e 60 anos"
    else:
        return "Maior de 60 anos"

def returnAnosDeEstudo(e):
    if(e == 17):
        return "Não Informado"
    return e - 1

def returnClasseSocial(r):
    salario_minimo = 1100
    if(r <= salario_minimo):
        return "E"
    elif(r <= salario_minimo*3):
        return "D"
    elif(r <= salario_minimo*5):
        return "C"
    elif(r <= salario_minimo*15):
        return "B"
    else:
        return "A"


df['Sexo'] = df['Sexo'].apply(lambda x: returnGender(x))
df['UF'] = df['UF'].apply(lambda x: returnUF(x))
df['Cor'] = df['Cor'].apply(lambda x: returnEthnicGroup(x))
df['Idade'] = df['Idade'].apply(lambda x: returnGruposDeIdade(x))
df['Anos de Estudo'] = df['Anos de Estudo'].apply(lambda x: returnAnosDeEstudo(x))
df['Renda'] = df['Renda'].apply(lambda x: returnClasseSocial(x))

# Entropia e Indice de Gini

São definidos as funções para cáculo de entropia e ganho de determinada que serão usados na construção da Árvore de Decisão.

In [ ]:
def calcularEntropia(df, coluna):
    values = df[coluna].unique()
    entropia = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        entropia += pi*np.log2(pi)
    return -entropia
    
def calcularGanho(df, resultado, coluna):
    values = df[coluna].unique()
    ganho = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        ganho += pi * calcularEntropia(df[df[coluna].eq(i)], resultado)
    ganho = calcularEntropia(df, resultado) - ganho
    return ganho

def calcularIndiceDeGiniParaValor(df, resultado, valor, coluna):
    gini = 0
    for value in df[resultado].unique():
        pi = len(df[df[coluna].eq(valor) & df[resultado].eq(value)])/len(df[df[coluna].eq(valor)])
        gini += pi * pi
    gini = 1 - gini
    return gini
    
def calcularIndiceDeGini(df, resultado, coluna):
    values = df[coluna].unique()
    gini = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        gini += calcularIndiceDeGiniParaValor(df, resulado, i, coluna) * pi
    
    return gini

# A árvore de decisão

A classe nó abaixo é a definição da arvóre, sendo a variável 'atributo' o atributo do dataset que representa aquele específico Nó, no caso do dataset em questão os possíveis atributos de um Nó são: ('Anos de Estudo', 'Sexo', 'Idade', 'Cor' e 'UF'), todo o nó final de uma arvóres terá como atributo um determinado valor da coluna que se procura prever, no caso do exemplo os atributos no ultimo nó de uma árvore podem ser ('A', 'B', 'C', 'D' e 'E') e a váriavel galho assumirá a string '.'<br>

A variável galhos da classe Nó, representa os próximos níveis da árvore a partir do corrente nó, a variável é um dicionário, cuja as chaves são os possíveis valores do atributo daquele nó, e os valores de cada uma das chaves do dicionário 'galhos', são os nós do seguinte nível da árvores.
Como exemplo uma variável galhos de um Nó, cujo atributo é 'Sexo', terá como valor:<br>
`{
    "Masculino": Proximo Nó,
    "Feminino": Proximo Nó
}`

---
O algoritmo de implementação da Árvore de Decisão, usa a variável funcao (definidas anteriormente, calcularGanho() e calcularIndiceDeGini()) para decidir qual a melhor coluna para caracterizar o primeiro nó da árvore, e constroi os seguintes nós a partir desse de forma recursiva com os critérios de parada sendo, o dataset se torna vazio, o dataframe ter somente um valor da coluna de resultado (no caso do exemplo, 'Renda'), e não existirem mais colunas para serem avalidas, desse forma a árvore retornará o nó inicial que por sua vez referencia todos os outros.

In [ ]:
class No:
    def __init__(self, atributo, galhos):
        self.atributo = atributo
        self.galhos = galhos
        
def construirArvoreDeDecisao(df, colunas, resultado, resultadoAnterior=0, funcao=calcularGanho, dfOriginal=df):
    if(len(df[resultado]) == 0):
        return No(resultadoAnterior, '.')
    elif(len(df[resultado].unique()) == 1):
        return No(df[resultado].iloc[0], '.')
    elif(len(colunas) == 0):
        return No(df[resultado].value_counts().index[0], '.')
    
    maiorGanhoDeColunas = -np.inf
    for i in colunas:
        atualGanho = funcao(df, resultado, i)
        if maiorGanhoDeColunas < atualGanho:
            colunaEscolhida = i
            maiorGanhoDeColunas = atualGanho
    
    galhos = {}
    for i in dfOriginal[colunaEscolhida].unique():
        galhos[i] = construirArvoreDeDecisao(df[df[colunaEscolhida].eq(i)], list(set(colunas) - set([colunaEscolhida])), resultado, df[resultado].value_counts().index[0], funcao, dfOriginal)
        
    return No(colunaEscolhida, galhos)

# Buscar na Àrvore

A função de buscar na árvore, desce pelos nó da árvore até encontrar o valor '.' na variável galhos, o que indica o final da árvore, tal busca é realizada de maneira recursiva.

---

A entrada para a função são a árvore e uma variável indivíduo, que é um dicionário cujas chaves são as colunas (com execeção da coluna resultado), e os valores são os valores que determinado indivíduo possui para aquelas colunas.

In [ ]:
def buscarNaArvore(arvore, individuo):
    if(arvore.galhos == '.'):
        return arvore.atributo
    else:
        return buscarNaArvore(arvore.galhos[individuo[arvore.atributo]], individuo)

# K-Fold Validation

Na seguinte célula são realizados 10 experimentos seguindo o 5-fold validation, são 5 experimentos de k-fold validation com k igual a 5 usando como função o cálculo de ganho e outros 5 usando como função o índice de gini.

---

O resultado de cada um dos experimentos é uma tabelas com linhas ['A', 'B', 'C', 'D', 'E', 'Total'] e colunas ['A', 'B', 'C', 'D', 'E', 'Total', 'Precisão', 'Reconhecimento'].<br>
As colunas referentes as classes e a coluna 'Total' representam todos os indivíduos do dataset que foram classificados daquela maneira, enquanto as linhas referentes as classes e a linha 'Total' representam todas os indivíduos do dataset, que foram classificados daquela maneira pela Árvore de Decisão. Como exemplo, o valor na coluna 'B' e linha 'C', representam todos os indivuos que são realmente da classe social 'B', mas que foram classificados como 'C' pela Árvore de Decisão. <br>

A coluna 'Precisão' do dataframe na linha 'A', é a proporção de todos os indivíduos que o algoritmo corretamente classificou como 'A' pela quantidade de indivíduos classificados, pelo algoritmo como A.<br>
A coluna 'Reconhecimento' do dataframe na linha 'A', é a proporção de todos os indivíduos que o algoritmo corretamente classificou como 'A' pela quantidade de indivíduos que são realmente da classe social 'A'.


In [ ]:
def kFoldValidation(k, df, funcao=calcularGanho):
    resto = len(df) % k
    resultadosExperimentos = []
    resultadosBusca = {}
    
    for i in range(0, k):
        df = df.sample(frac=1)
        teste = df.iloc[int((i/k)*len(df)):int(((i+1)/k)*len(df) + 1)]
        treinamento = df.iloc[:int((i/k)*len(df))].append(df.iloc[int(((i+1)/k)*len(df) + 1):])
        arvore = construirArvoreDeDecisao(treinamento, ['UF', 'Sexo', 'Idade', 'Cor', 'Anos de Estudo'], 'Renda', funcao)
        for r in df['Renda'].unique():
            resultadosBusca[r] = {}
            for f in df['Renda'].unique():
                resultadosBusca[r][f] = 0
        for e in teste.values:
            v = buscarNaArvore(arvore, {'UF': e[0], 'Sexo': e[1], 'Idade': e[2], 'Cor': e[3], 'Anos de Estudo': e[4]})
            resultadosBusca[e[5]][v] += 1
        
        dfAux = pd.DataFrame.from_dict(resultadosBusca)
        dfAux = dfAux.sort_index(ascending=True).sort_index(axis=1,ascending=True)
        dfAux['Total'] = dfAux.sum(axis=1)
        dfAux.loc['Total'] = dfAux.sum(axis=0)
        dfAux['Precisão'] = np.divide(np.diag(dfAux), dfAux['Total'])
        dfAux['Precisão']['Total'] = np.sum(np.diag(dfAux)[:-1])/dfAux['Total'].loc['Total']
        dfAux['Reconhecimento'] = np.divide(np.diag(dfAux), dfAux.loc['Total'][:-1])
        dfAux['Reconhecimento']['Total'] = np.sum(np.diag(dfAux)[:-1])/dfAux['Total'].loc['Total']
        resultadosExperimentos.append(dfAux.copy())
    
    return resultadosExperimentos

kfoldResultsEntropia = kFoldValidation(5, df)
kfoldResultsIndiceDeGini = kFoldValidation(5, df, calcularIndiceDeGini)

# Resultados Entropia

Resultados dos experimento usando a função de ganho como parâmetro para escolher as colunas

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[0])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[1])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[2])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[3])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[4])

# Resultados Indice de Gini

Resultados dos experimento usando o Indice de Gini como parâmetro para escolher as colunas

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[0])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[1])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[2])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[3])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[4])

A seguir será construido o dataframe para trabalhar com o método de Árvore de decisão da biblioteca sklearn

In [ ]:
sklearnDf = pd.get_dummies(df[['Sexo','UF','Cor','Idade', 'Anos de Estudo']])
sklearnDf

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy")

clf = clf.fit(sklearnDf[:15368*4], df.Renda[:15368*4])

# K-Fold Validation Scikit
Aqui temos a implementação do método k-fold validation, como no exemplo anterior, com a exceção de que, dessa vez usamos o método com a ávore construida pela biblioteca do scikit learn.

---

Apesar de usarmos um diferente método de construção de árvore o retorno dos experimentos é o mesma tabela.

In [ ]:
def kFoldValidationScikit(k, df, criterio="entropy"):
    resto = len(df) % k
    resultadosExperimentos = []
    resultadosBusca = {}
    
    for i in range(0, k):
        df = df.sample(frac=1)
        teste = df.iloc[int((i/k)*len(df)):int(((i+1)/k)*len(df) + 1)]
        treinamento = df.iloc[:int((i/k)*len(df))].append(df.iloc[int(((i+1)/k)*len(df) + 1):])
        arvore = DecisionTreeClassifier(criterion=criterio).fit(pd.get_dummies(treinamento[['Sexo','UF','Cor','Idade', 'Anos de Estudo']]), treinamento['Renda'])
        for r in df['Renda'].unique():
            resultadosBusca[r] = {}
            for f in df['Renda'].unique():
                resultadosBusca[r][f] = 0
        prediction = clf.predict(pd.get_dummies(teste[['Sexo','UF','Cor','Idade', 'Anos de Estudo']]))       
        for e in range(len(prediction)):
            resultadosBusca[teste['Renda'].iloc[e]][prediction[e]] += 1 
        
        dfAux = pd.DataFrame.from_dict(resultadosBusca)
        dfAux = dfAux.sort_index(ascending=True).sort_index(axis=1,ascending=True)
        dfAux['Total'] = dfAux.sum(axis=1)
        dfAux.loc['Total'] = dfAux.sum(axis=0)
        dfAux['Precisão'] = np.divide(np.diag(dfAux), dfAux['Total'])
        dfAux['Precisão']['Total'] = np.sum(np.diag(dfAux)[:-1])/dfAux['Total'].loc['Total']
        dfAux['Reconhecimento'] = np.divide(np.diag(dfAux), dfAux.loc['Total'][:-1])
        dfAux['Reconhecimento']['Total'] = np.sum(np.diag(dfAux)[:-1])/dfAux['Total'].loc['Total']
        resultadosExperimentos.append(dfAux.copy())
    
    return resultadosExperimentos

kfoldResultsEntropia = kFoldValidationScikit(5, df, 'entropy')
kfoldResultsIndiceDeGini = kFoldValidationScikit(5, df, 'gini')

# Resultados Entropia

Resultados dos experimento usando a função de ganho como parâmetro para escolher as colunas

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[0])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[1])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[2])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[3])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsEntropia[4])

# Resultados Indice de Gini

Resultados dos experimento usando o Indice de Gini como parâmetro para escolher as colunas

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[0])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[1])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[2])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[3])

In [ ]:
pd.DataFrame.from_dict(kfoldResultsIndiceDeGini[4])

# Aplicando o post-prunning 

As seguinte células, aplicam o método da biblioteca sklearn na árvore construida de cost_complexity_prunning_path, que nos retorna uma lista de valores de 0 a 1, tais valores que representam o quanto da árvore deve ser cortado para um melhor desempenho

In [ ]:
path = clf.cost_complexity_pruning_path(pd.get_dummies(df[['Sexo','UF','Cor','Idade', 'Anos de Estudo']]), df['Renda']) 
alphas = path['ccp_alphas']

Depois de construida a lista de alphas realizamos o teste para cada um deles com as árvores diferentes árvores para cada, na célula seguinte

In [ ]:
accuracy_train, accuracy_test = [], []
for i in alphas[alphas>0]:
    i = i if i >= 0 else i*-1
    
    tree = DecisionTreeClassifier(ccp_alpha=i)
    
    tree.fit(sklearnDf[:15368*4], df.Renda[:15368*4])
    y_train_pred = tree.predict(sklearnDf[:15368*4])
    y_test_pred = tree.predict(sklearnDf[15368:])
    
    accuracy_train.append(metrics.accuracy_score(df.Renda[:15368*4], y_train_pred))
    accuracy_test.append(metrics.accuracy_score(df.Renda[15368:], y_test_pred))

# Desempenho do Post-Pruning
O seguinte gráfico nos mostra o desempenho do post-pruning para os exemplos de treino e de teste, o gráfico é um plot da AcuráciaXalpha de corte da Árvore.<br>
Pelo gráfico podemos perceber que o melhor desempenho do algoritmo aconteceu, quando o nosso valor para o alpha da árvore era igual a 0, ou seja, uma possível poda na Árvore pioraria o desempenho das suas previsões

In [ ]:
sns.set()
plt.figure(figsize=(14,7))
sns.lineplot(y=accuracy_train, x=alphas[alphas>0])
sns.lineplot(y=accuracy_test, x=alphas[alphas>0])
plt.xticks(ticks=np.arange(0.00, 0.25, 0.01))
plt.show()